In [1]:
import dremio_client.lib as dlib

import pandas as pd
import h3pandas
import sqlserver_client.db_connector as dc
from datetime import datetime
import time


In [2]:
def getTag29Trnx(env):
    if env == 'UAT':
        engine = dc.getFastEngineUAT()
    elif env == 'PROD':
        engine = dc.getFastEnginePROD()
    else:
        engine = dc.getFastEngineLocal()

    query = f"""
        SELECT 
            date_trunc('month',created_at) as dt,
            src_name, MerchantName, 
            COUNT(*) AS total_trnx, 
            SUM(CASE WHEN ccy = 'KHR' THEN amount/4000.00 ELSE amount END) as total_usd_amount
        FROM "DataScience_DB"."General_DB".dbo.bk_outgoing
        where date_trunc('month',created_at) = '2023-12-01'
        and type = 'QR' AND MerchantType = '29'
        AND src_name <> MerchantName
        GROUP BY 1,2,3
        ORDER BY 3
        LIMIT 500000
        OFFSET 500000
    """

    np_df = dlib.simple_query(query)

    return np_df

In [3]:
def getAllAccountName(env):
    if env == 'UAT':
        engine = dc.getFastEngineUAT()
    elif env == 'PROD':
        engine = dc.getFastEnginePROD()
    else:
        engine = dc.getFastEngineLocal()

    query = f"""
        SELECT DISTINCT CIF, AC_NAME
        FROM dwh.FCCBOREP.VW_CUSTACC
        ORDER BY 1
    """

    np_df = dlib.simple_query(query)

    return np_df


In [4]:
tag29_df = getTag29Trnx('UAT')

AttributeError: module 'sqlserver_client.db_connector' has no attribute 'getFastEngineUAT'

In [6]:
tag29_df.head()

,dt,src_name,MerchantName,total_trnx,total_usd_amount
0,2023-12-01,SREYNAN SOR,,1,10.000000000
1,2023-12-01,BONDOL KEAN,,1,5.000000000
2,2023-12-01,VOUCHLY VENG,,1,40.500000000
3,2023-12-01,SREYMOM KAY,,1,50.000000000
4,2023-12-01,LEAKHENA CHIN,,1,92.320000000


In [7]:
tag29_df['MerchantName'] = tag29_df['MerchantName'].str.upper()
tag29_df['MerchantName'] = tag29_df['MerchantName'].str.strip()

In [8]:
print(tag29_df.count())

print(tag29_df['total_trnx'].sum())

print(tag29_df['MerchantName'].nunique())

dt                  3160803
src_name            3160803
MerchantName        3160803
total_trnx          3160803
total_usd_amount    3160803
dtype: int64
5912032
997060


In [9]:
tag29_df.head()

,dt,src_name,MerchantName,total_trnx,total_usd_amount
0,2023-12-01,SREYNAN SOR,,1,10.000000000
1,2023-12-01,BONDOL KEAN,,1,5.000000000
2,2023-12-01,VOUCHLY VENG,,1,40.500000000
3,2023-12-01,SREYMOM KAY,,1,50.000000000
4,2023-12-01,LEAKHENA CHIN,,1,92.320000000


In [10]:
unique_merchantname = pd.DataFrame({'Unique_MerchantName': tag29_df['MerchantName'].drop_duplicates()})

In [11]:
unique_merchantname.head(10)

,Unique_MerchantName
0,
58,CHAN SIPANA
59,RITH SEYHA
60,រី រ៉ន
61,សេង វណ្ណា
62,SAY BUNTHEA
63,TAING KEMTRY
65,សោម ភត្រ្តា
66,CHHAIVAT DOEURN
67,CHHUN VANNET


In [12]:
unique_merchantname.count()

Unique_MerchantName    997060
dtype: int64

In [13]:
aba_customers = getAllAccountName('UAT')

In [21]:
aba_customers.head()

,AC_NAME
0,\tPENG KHEANG AND CHOU MUYSORTH
1,\tRO SPHEARA
2,CAMBODIA CANAAN ORGANIZATION
3,MEL PHEAROM AND HOK BORA
4,TES TITSORYA


In [17]:
aba_customers.count()

AC_NAME    object
dtype: object

In [129]:
from rapidfuzz import fuzz as rf
# Token Sort Ratio is useful when word order might vary
def isABACustomer(str1,str2):
    return rf.token_sort_ratio(str1, str2)
    # return fuzz.token_sort_ratio(row['src_name'], row['MerchantName'])

In [130]:
for index_df1, row_df1 in unique_merchantname.iterrows():
    print('index_df1: ',index_df1)
    max_similarity_score = 0
    for index_df2, row_df2 in aba_customers.iterrows():        
        if index_df2 % 500000 == 0:
            print('index_df2: ',index_df2)
        # Apply the compare_strings function to each pair of rows
        similarity_score = isABACustomer(row_df1['Unique_MerchantName'], row_df2['AC_NAME'])
        # Update the 'Similarity Score' column if similarity score is higher
        if similarity_score >= 95:
            max_similarity_score = similarity_score 
            break
    unique_merchantname.at[index_df1, 'Max_Similarity_Score'] = max_similarity_score
               

index_df1:  0
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df2:  2000000
index_df1:  58
index_df2:  0
index_df1:  59
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df1:  60
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df2:  2000000
index_df1:  61
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df2:  2000000
index_df1:  62
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df1:  63
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df2:  2000000
index_df1:  65
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df2:  2000000
index_df1:  66
index_df2:  0
index_df2:  500000
index_df2:  1000000
index_df2:  1500000
index_df2:  2000000
index_df1:  67
index_df2:  0
index_df1:  68
index_df2:  0
index_df2:  500000
index_df1:  73
index_df2:  0
index_df2:  500000
index_df1:  74
index_df

KeyboardInterrupt: 

In [ ]:
unique_merchantname